In [ ]:
import numpy as np
import math
import os
import pandas as pd
import matplotlib
from scipy.integrate import ode as ode
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import xlsxwriter
from scipy.optimize import curve_fit
import seaborn as sns
sns.set_style('whitegrid')


In [ ]:
## Adjust ligand concentration with the Koff and Kon values in order to shift the DR-curves along the x-axis, 
# because of the change in the Kd values of the ligand caused by the changed Koff.Kon value from the default values (koff=0.0003,kon=0.3)
def adjust_ligand_ccs(ligand_ccs,koff,kon):
    Kd_ratio = (koff/ kon) / (0.0003 / 0.3)
    ligands = ligand_ccs * Kd_ratio
    return (ligands)

In [ ]:
def set_time_range(start,stop):
    time = pd.read_excel('Time (s).xlsx')
    time = time.loc[start*10:stop*10+0.1, 0].tolist()
    return time

class data_evaluation(object):
    def __init__(self):
        self.AR_dicts, self.AR_ccs, self.IP3R_dicts, self.IP3R_ccs, self.phys_dicts, self.phys_ccs,self.RA_dicts,self.RA_ccs= {}, {}, {}, {}, {}, {},{},{}
        """
        contains data in form xx_ccs={kon_xx={koff_xx:[list of used ligand ccs]
        contains data in form  xx_dicts={kon_xx={koff_xx:[list of dataframes containing the calculated concentration values of the 46 modelled molecules 
        at different ligand concentrations], loaded from '/Simulation values/directory_names[]/koff xx - kon xx.xlsx' excel files.  
        for example: xx_dicts={kon_xx={koff_xx:[df_10umol_ligand,df_100umol_ligand,df_1mmol_ligand, etc...]
        """
        self.dict_list = [self.AR_dicts, self.IP3R_dicts, self.phys_dicts,self.RA_dicts]
        self.ccs_list = [self.AR_ccs, self.IP3R_ccs, self.phys_ccs,self.RA_ccs]


    #load calculated concentration values of the 46 modelled molecules into a dataframe, then create a list of the dataframes
    def load_data(self,directory,kon,koff,values_dict):
        for file in os.listdir(directory+'/'):
            filename=os.path.join(directory,file)
            if file=='koff '+str(koff)+' - '+'kon '+str(kon)+'.xlsx':
                xl=pd.ExcelFile(filename)
                ccs_list=[]
                for name in xl.sheet_names:     #xl.sheet_names is a list of the ligand concentrations
                    simulated_ccs=pd.read_excel(xl,sheet_name=name,dtype=np.float32)
                    ccs_list.append(simulated_ccs)
                values_dict['kon_' + str(kon)]['koff_' + str(koff)]=ccs_list

    # load the lists of dataframes into the corresponding self.xx_dicts, add them to self.dict_list
    def load_data_into_dicts(self, directory_names,dict_list,internalization=True):
        for directory, values_dict in zip(directory_names,dict_list):
            for kon in kon_perturbation:
                values_dict['kon_' + str(kon)] = {}
                for koff in koff_perturbation:
                    if internalization==False:
                        filedir='Model/Simulation values/No internalization/' + directory
                    elif internalization==True:
                        filedir = 'Model/Simulation values/' + directory
                    self.load_data(filedir, kon, koff, values_dict)

    # load the lists of ligand ccs into the corresponding self.xx_ccs, add them to self.ccs_list
    def load_ccs_into_dicts(self,kon_perturbation,koff_perturbation,cc_dict):
        cc_dict={}
        for kon in kon_perturbation:
            cc_dict['kon_'+str(kon)]={}
            fname = 'Model/Ligand concentrations/kon_'+str(kon)+'.xlsx'
            df=pd.read_excel(fname)
            for koff in koff_perturbation:
                cc_dict['kon_' + str(kon)]['koff_'+str(koff)]=df['koff_'+str(koff)].tolist()
        return cc_dict



    def plot(self,time,values,molecules,ax=False):
        if ax:
            ax.plot(time,np.sum(values.loc[:,molecules],1))

    def dose_response(self,label,values, molecules,time,time_point):
        time_ccs = pd.DataFrame({label: list(np.sum(values.loc[:, molecules], 1)), 'Time': list(time)})
        value = time_ccs.loc[time_point,label]
        return value

    def DR_curvefit(self, DR_points, ligands,kon=False,kon_value=False,ax=False,molecule=False,timedir=False,label=False,fig=False,legend_title=False):
        def func(ligands, logec50):
            return max(DR_points) / (1 + 10 ** (logec50 - np.log10(ligands)))
        popt, pcov = curve_fit(func, ligands, DR_points,bounds=(-19,-1))
        if label and fig is not None:
            ligandrange = np.linspace(np.log10(ligands[0]), np.log10(ligands[-1]), 100)
            color = next(ax[kon_value]._get_lines.prop_cycler)['color']
            ax[kon_value].plot((ligandrange), func(10 ** ligandrange, popt), c=color)
            ax[kon_value].plot(np.log10(ligands), DR_points, '.',label=label, c=color)
            if legend_title:
                ax[kon_value].legend(title=legend_title)
            else:
                ax[kon_value].legend()
            self.subplot_labels(ax[kon_value],molecule,timedir,kon)
        else:
            logec50 = (popt)
            return logec50

    def subplot_labels(self,ax,ylabel,title,kon_value):
        ax.set_xlabel('Ligand concentration M')
        ax.set_ylabel(ylabel)
        ax.set_title(title +' Kon=' + str(kon_value))

    def write_toexcel(self,fname,data_list,sheet_name):
        writer = pd.ExcelWriter(fname, engine='xlsxwriter')
        for a in range(len(data_list)):
            data_list[a].to_excel(writer, sheet_name=sheet_name[a])
        writer.save()


# draw time-concentration curves of assessed  molecule (i.e.: receptor-arrestin complex, IP3, IP3-sensor) at different kon-koff values
def draw_time_curves(values_dict, cc_dict, molecule, dir, kon_perturbation, koff_perturbation_short,time):
    fig, ax = plt.subplots(3, 3)
    fig.canvas.set_window_title(dir)
    for a in range(len(kon_perturbation)):
        for b in range(len(koff_perturbation_short)):

            if molecule == "Arrestin-Receptor":  # leave the first 3 ligand concentrations out, because calculated concentrations are at minimum there
                values_df = values_dict['kon_' + str(kon_perturbation[a])]['koff_' + str((koff_perturbation_short[b]))][
                            3:]
                cc_df = cc_dict['kon_' + str(kon_perturbation[a])]['koff_' + str((koff_perturbation_short[b]))][3:]
            else:  # leave the last 3 ligand concentrations out, because calculated concentrations are at maximum there
                values_df = values_dict['kon_' + str(kon_perturbation[a])]['koff_' + str((koff_perturbation_short[b]))][
                            1:-2]
                cc_df = cc_dict['kon_' + str(kon_perturbation[a])]['koff_' + str((koff_perturbation_short[b]))][1:-2]

            for df in values_df:
                data.plot(time, df, molecules_dict[molecule], ax[a, b])
                ax[a, b].legend(np.log10(cc_df), title="log[Ligand]")
                ax[a, b].set_xlabel('Time (s)')
                ax[a, b].set_ylabel(molecule)
                ax[a, b].set_title('Koff:' + str(koff_perturbation_short[b]) + ' - Kon:' + str(kon_perturbation[a]))
    plt.tight_layout()
    plt.gcf().set_size_inches((width / 100, height / 100))
    return plt




# calculate dose-response curves of Arrestin-receptor and IP3-sensor at 3 different experimental designs: Arrestin overexpr., IP3-sensor overexpr., physiological condition

def calculate_DR_points(time_point,timedir, dict_list, ccs_list, directory_names, DR_molecules_list,EC50_list,internalization=True):
    for values_dict, cc_dict, dir in zip(dict_list, ccs_list, directory_names):
        for a in range(len(kon_perturbation)):
            for b in range(len(koff_perturbation)):

                # compute values of Arrestin-Receptor, Sensor-IP3,IP3 DR curves
                DR_points = np.array([])
                for df in values_dict['kon_' + str(kon_perturbation[a])]['koff_' + str(koff_perturbation[b])]:  # accessing list of dataframes of the values
                    for DR_molecule, mol_number in zip(DR_molecules_list, range(len(DR_molecules_list))):
                        response = data.dose_response(DR_molecule, df, molecules_dict[DR_molecule], time, time_point)
                        DR_points = np.append(DR_points, response)
                DR_points = np.reshape(DR_points, (len(cc_dict['kon_' + str(kon_perturbation[a])]['koff_' + str(koff_perturbation[b])]),
                            len(DR_molecules_list)))  # reshaping to fit dataframe format
                DR_points_df = pd.DataFrame(DR_points, index=cc_dict['kon_' + str(kon_perturbation[a])][
                            'koff_' + str(koff_perturbation[b])], columns=DR_molecules_list)


                # compute EC50 values of Arrestin-Receptor, Sensor-IP3,IP3 DR curves
                EC50_points = []
                for mol in DR_molecules_list:
                    logec50 = data.DR_curvefit(DR_points_df.loc[:, mol].tolist(),cc_dict['kon_' + str(kon_perturbation[a])]['koff_' + str(koff_perturbation[b])])
                    EC50_points.append(10 ** logec50)
                EC50_points_df = pd.DataFrame(EC50_points, index=DR_molecules_list,columns=EC50_list)  # converting to Dataframe
                EC50_points_df = EC50_points_df.T  # transposing to proper layout

                # saving created dataframes to excel files
                df_list = [DR_points_df, EC50_points_df]
                sheet_names = ["DR-curves", "EC50 values"]
                if internalization == False:
                    fname = "Model/Simulation values/No internalization/" + dir + "/DR points/" + timedir + "/koff " + str(
                        koff_perturbation[b]) + " - kon " + str(kon_perturbation[a]) + ".xlsx"
                elif internalization == True:
                    fname = "Model/Simulation values/" + dir + "/DR points/" + timedir + "/koff " + str(
                        koff_perturbation[b]) + " - kon " + str(kon_perturbation[a]) + ".xlsx"
                data.write_toexcel(fname, df_list, sheet_names)


# Plotting DR-curves of Arrestin-receptor and IP3-sensor at 3 different experimental designs: Arrestin overexpr., IP3-sensor overexpr., physiological condition
def draw_DR_curves(cc_dict, dir, internalization=True):
    # determine, which molecules will be plotted
    molecules_to_plot = []
    if dir == "Arrestin-Receptor overexpression":
        molecules_to_plot = ["Arrestin-Receptor", "IP3"]
    elif dir == "IP3-Receptor overexpression":
        molecules_to_plot = ["SensorIP3", "IP3"]
    elif dir == 'Physiological system':
        molecules_to_plot = ["Arrestin-Receptor", "IP3"]

    # plotting the DR curves
    for timedir in time_directory_names:
        fig1, fig2 = plt.figure(), plt.figure()
        ax1, ax2 = fig1.subplots(nrows=1, ncols=3), fig2.subplots(nrows=1, ncols=3)
        ax_list = [ax1, ax2]
        fig_list = [fig1, fig2]
        for ax, fig, mol_plot in zip(ax_list, fig_list, molecules_to_plot):
            for a in range(len(kon_perturbation)):
                for b in range(len(koff_perturbation)):
                    if internalization == False:
                        fname_load = "Model/Simulation values/No internalization/" + dir + "/DR points/" + timedir + "/koff " + str(
                            koff_perturbation[b]) + " - kon " + str(kon_perturbation[a]) + ".xlsx"
                        fname_savefig = 'Model/Figures/DR curves/' + dir + ' ' + timedir + ' ' + mol_plot + 'no internalization.png'
                    elif internalization == True:
                        fname_load = "Model/Simulation values/" + dir + "/DR points/" + timedir + "/koff " + str(
                            koff_perturbation[b]) + " - kon " + str(kon_perturbation[a]) + ".xlsx"
                        fname_savefig = 'Model/Figures/DR curves/' + dir + ' ' + timedir + ' ' + mol_plot + '.png'
                    DR_points_df = pd.read_excel(fname_load, sheet_name="DR-curves")

                    values_list = DR_points_df[mol_plot].tolist()
                    cc_list = cc_dict['kon_' + str(kon_perturbation[a])]['koff_' + str((koff_perturbation[b]))]
                    data.DR_curvefit(values_list, cc_list, str(kon_perturbation[a]), a, ax, mol_plot, timedir,
                                     koff_perturbation[b], fig)
                    fig.tight_layout()
                    fig.set_size_inches((width / 100, height / 100))
            fig.savefig(fname_savefig, dpi=100)
            plt.close(fig)


#Emax-EC50 and Emax-Kd curves

def draw_Emax_EC50_plot(timedir,molecules_list,directory_names):
    for molecule,dir in zip(molecules_list,directory_names):
        fig = plt.figure()
        ax = fig.subplots(nrows=2, ncols=1)
        fig.canvas.set_window_title(dir+" "+timedir)

        #computing Emax, logEC50 and logKd values
        Emax_list, LogEC50_list, LogKd_list = [[]], [[]], [[]]
        for a in range(len(kon_perturbation)):
            Emax_list.append([])
            LogKd_list.append([])
            LogEC50_list.append([])
            for b in range(len(koff_perturbation)):
                LogKd_list[a].append(np.log10(koff_perturbation[b] / kon_perturbation[a]))
                fname='Model/Simulation values/'+dir+'/DR points/'+timedir+'/koff '+str(koff_perturbation[b])+' - kon '+str(kon_perturbation[a])+'.xlsx'
                xl=pd.ExcelFile(fname)
                emax_df=pd.read_excel(xl,sheet_name=xl.sheet_names[0])
                ec50_df=pd.read_excel(xl,sheet_name=xl.sheet_names[1])
                Emax_list[a].append(emax_df[molecule].max())
                LogEC50_list[a].append(np.log10(ec50_df.loc[:,molecule]))

            #plotting data
            ax[0].plot(LogEC50_list[a], Emax_list[a], '.', markersize=9)
            ax[1].plot(LogKd_list[a], Emax_list[a], '.', markersize=9)
        ax[0].set_title(dir + " " + timedir, fontsize=10)
        ylabels=['Log EC50','Log Kd']
        for num in range(len(ylabels)):
            ax[num].set_xlabel(ylabels[num], size=10)
            ax[num].set_ylabel(molecule + "\n" + "(Emax)", size=10)
            ax[num].legend(kon_perturbation)
            ax[num].set_ylim([0, None])
        plt.tight_layout()
        plt.gcf().set_size_inches((width / 200, height / 200))
        plt.savefig("Model/Figures/Emax-EC50 curves/" + dir + " " + timedir + ".png", dpi=200)
        plt.close(fig)


#compute bias factors and plot them on heatmaps

#selecting the two pathways that get compared for bias
def selecting_pathways_for_bias(dir,pathways):
    if dir=='Arrestin-Receptor overexpression' or dir=='Physiological system' or dir=='Receptor-Arrestin overexpression':
        pathways = ["Arrestin-Receptor", "IP3"]
    elif dir=='IP3-Receptor overexpression':
        pathways = ["Arrestin-Receptor", "SensorIP3"]
    return pathways

#computing emax and ec50 values at a given kon-koff pair, return them as dictionaries
def compute_emax_ec50(dir,timedir,kon,koff,pathway):
    filename="Model/Simulation values/"+dir+"/DR points/"+timedir+"/koff "+str(koff)+" - kon "+str(kon)+".xlsx"
    excelfile = pd.ExcelFile(filename)
    emax_df=pd.read_excel(excelfile, sheet_name="DR-curves")
    ec50_df=pd.read_excel(excelfile, sheet_name="EC50 values")
    emax_dict,ec50_dict={},{}
    for path in pathway:
        emax=emax_df[path].max()
        emax_dict[path]=emax
        ec50=ec50_df[path].values[0]
        ec50_dict[path]=ec50
    return emax_dict,ec50_dict

def create_heatmap(bias_factor_df,dir,ylabel):#plotting the computed bias factors at different kon-koff pairs as heatmaps
    fig = plt.figure()
    ax=sns.heatmap(bias_factor_df,annot=True,cmap="YlGnBu", fmt='.4f',cbar_kws={'ticks': [-1,-0.5,0,0.5,1.0]},vmin=-1,vmax=1)
    ax.set_title(dir)
    ax.set_xlabel("Time (s)")
    ax.set_ylabel(ylabel)
    sns.set(font_scale=0.6)
    plt.gcf().set_size_inches((width / 200, height / 200))
    plt.savefig("Model/Figures/Bias factor heatmap/"+dir+" "+ylabel+".png",dpi=200)
    plt.close(fig)

def compute_bias_factor(koff,kon,dir,pathways,ylabel,bias_factor_dict):
    bias_factor_list = []
    for timedir in time_directory_names:
        #computing emax and ec50 values at reference kon-koff pair (0.3 and 0.003) and creating emax/ec50 quotient necessary for the bias factor
        reference_emax_dict, reference_ec50_dict = compute_emax_ec50(dir, timedir, 0.3, 0.003, pathways)
        reference_quotient=(reference_emax_dict[pathways[0]]/reference_ec50_dict[pathways[0]])/(reference_emax_dict[pathways[1]]/reference_ec50_dict[pathways[1]])

        #computing emax and ec50 values at different kon-koff pairs,creating emax/ec50 quotient necessary for the bias factor and computing the bias factor
        emax_dict, ec50_dict = compute_emax_ec50(dir, timedir, kon, koff, pathways)
        ligand_quotient=(emax_dict[pathways[0]]/ec50_dict[pathways[0]])/(emax_dict[pathways[1]]/ec50_dict[pathways[1]])
        bias_factor=math.log10(ligand_quotient/reference_quotient)
        bias_factor_list.append(bias_factor)
    if ylabel=="Koff":
        bias_factor_dict[koff]=bias_factor_list
    elif ylabel=="Kon":
        bias_factor_dict[kon] = bias_factor_list
    bias_factor_df=pd.DataFrame.from_dict(bias_factor_dict,orient='index',columns=time_directory_names)
    create_heatmap(bias_factor_df,dir,ylabel)




#calculating bias factors of 2 pathways from different experimental designs .
#Pathway 1: AR-overexpression arrestin-receptor, Pathway 2: IP3 overexpression IP3 sensor.
pathways=[["Arrestin-Receptor"], ["SensorIP3"]]
def compute_bias_factor_AR_IP3_OE(koff,kon,ylabel,bias_factor_dict):
    bias_factor_list = []
    for timedir in time_directory_names:
        #computing the arrestin-receptor emax-ec50 values from AR-overexpr. and IP3 sensor emax-ec50 from IP3 overexpression
        emax_dicts,ec50_dicts,reference_emax_dicts,reference_ec50_dicts,={},{},{},{}
        for dir,pathway in zip(directory_names[:-1],pathways):
            reference_emax_dict, reference_ec50_dict = compute_emax_ec50(dir, timedir, 0.3, 0.003, pathway)
            reference_emax_dicts[pathway[0]] = list(reference_emax_dict.values())
            reference_ec50_dicts[pathway[0]]= list(reference_ec50_dict.values())
            emax_dict, ec50_dict=compute_emax_ec50(dir,timedir,kon,koff,pathway)
            emax_dicts[pathway[0]]=list(emax_dict.values())
            ec50_dicts[pathway[0]]=list(ec50_dict.values())

        reference_quotient=(reference_emax_dicts[pathways[0][0]][0]/reference_ec50_dicts[pathways[0][0]][0])/(reference_emax_dicts[pathways[1][0]][0]/reference_ec50_dicts[pathways[1][0]][0])
        ligand_quotient = (emax_dicts[pathways[0][0]][0] /ec50_dicts[pathways[0][0]][0])/(emax_dicts[pathways[1][0]][0] /ec50_dicts[pathways[1][0]][0])
        bias_factor = math.log10(ligand_quotient / reference_quotient)
        bias_factor_list.append(bias_factor)
    if ylabel == "Koff":
        bias_factor_dict[koff] = bias_factor_list
    elif ylabel == "Kon":
        bias_factor_dict[kon] = bias_factor_list
    bias_factor_df = pd.DataFrame.from_dict(bias_factor_dict, orient='index', columns=time_directory_names)
    create_heatmap(bias_factor_df, "Arrestin-IP3 sensor overexpression", ylabel)





#Arrestin-receptor and IP3 Emax values simulated with different K values, depicted as boxplots.

def compute_emax(fname,time,time_point):
    xl = pd.ExcelFile(fname)
    DR_points = []
    for name in xl.sheet_names:
        df = pd.read_excel(xl, sheet_name=name)
        response = data.dose_response(molecule, df, molecules_dict[molecule], time, time_point)
        DR_points.append(response)
    return max(DR_points)


def k_perturbation_effects(cellular_model,k_values_dir,koff,kon,molecule,time,time_point,emax_pathway_dict):
    emax_list=[]
    for factor in factor_list:
        fname=('Model/Simulation with K values perturbation/Simulated values/' + str(cellular_model) + '/'+k_values_dir+'/'+str(factor)+' koff '
                    + str(koff) + " - kon " + str(kon) + ".xlsx")
        emax=compute_emax(fname,time,time_point)
        emax_list.append(emax)
    emax_pathway_dict[k_values_dir]=emax_list
    emax_pathway_df=pd.DataFrame.from_dict(emax_pathway_dict)
    emax_pathway_df['Factors']=factor_list
    emax_pathway_df.set_index('Factors',inplace=True)
    return emax_pathway_df


def boxplot(perturbations_df,cellular_model,molecule,timedir,kon=None,koff=None):
    palette = sns.color_palette("ch:2.5,-.2,dark=.3",n_colors=len(perturbations_df.index[2:]))
    df=perturbations_df.iloc[2:].melt(id_vars="Factors",var_name="Perturbations",value_name="Values")
    df.Factors=df.Factors.round(2)
    ax=sns.swarmplot(x="Perturbations",y="Values",data=df,hue="Factors",size=9,palette=palette)
    ax = sns.boxplot(data=perturbations_df.iloc[2:, 1:],showcaps=True,boxprops={'facecolor':'None'})
    ax.tick_params(axis='x', labelsize=15)
    ax.tick_params(axis='y',labelsize=15)
    ax.set_ylabel(molecule + ' Emax',fontsize=20)
    ax.set_xlabel("")


    plt.gcf().set_size_inches((width / 100, height / 100))
    if kon is not None and koff is not None:
        ax.set_title(cellular_model + ' ' + molecule + ' '+timedir+ ' kon ' + str(kon) + ' ' + 'koff ' + str(koff))
        plt.tight_layout()
        plt.savefig("Model/Figures/K values perturbation plots/" + cellular_model + '/' + molecule + ' ' +timedir+ ' kon ' + str(kon) + ' ' + 'koff ' + str(koff) + ".png", dpi=100)
    elif kon==None and koff==None:
        ax.set_title(cellular_model + ' ' + molecule+' '+timedir)
        plt.tight_layout()
        plt.savefig("Model/Figures/K values perturbation plots/" + cellular_model + '/High-Low affinity ratio ' + molecule+' '+timedir+".png", dpi=100)
    plt.cla()


#calculating the emax values at different K values, and saving them in dataframes necessary for the boxplots
def calculate_emax_at_diff_kvalues():
    for time_point,timedir in zip(time_points,time_directory_names):
        for molecule in molecules_list:
            emax_pathway_dict = {}
            for cellular_model in cellular_models:
                df_list=[]
                for koff, kon in zip(koff_values, kon_values):
                    fig = plt.figure()
                    for k_values_dir in k_values_directories:
                        perturbations_df=k_perturbation_effects(cellular_model, k_values_dir,koff, kon, molecule,time,time_point,emax_pathway_dict)
                    fname=('Model/Simulation with K values perturbation/Dataframes for boxplots/'+cellular_model+'/' + molecule+' '+timedir+ ' kon ' + str(kon) + ' ' + 'koff ' + str(koff)+'.xlsx')
                    perturbations_df.to_excel(fname,engine='xlsxwriter')
                    df_list.append(perturbations_df)

                #division of the emax values of the high affinity ligand with the emax values of the low affinity ligand
                divided_df=df_list[1].div(df_list[0])
                fname =('Model/Simulation with K values perturbation/Dataframes for boxplots/' + cellular_model +'/'+molecule+' '+timedir+' High-Low affinity ratio.xlsx')
                divided_df.to_excel(fname,engine='xlsxwriter')


#creating and saving the boxplots
def create_save_boxplots(koff_values,kon_values,dir,molecule,timedir):
    fname=('Model/Simulation with K values perturbation/Dataframes for boxplots/' + dir +'/'+molecule+' '+timedir+' High-Low affinity ratio.xlsx')
    divided_df=pd.read_excel(fname)
    boxplot(divided_df, dir, molecule, timedir)
    for koff, kon in zip(koff_values, kon_values):
                fname = ('Model/Simulation with K values perturbation/Dataframes for boxplots/' + dir + '/' + molecule + ' ' + timedir + ' kon ' +
                         str(kon) + ' ' + 'koff ' + str(koff) + '.xlsx')
                perturbations_df=pd.read_excel(fname)
                boxplot(perturbations_df, dir, molecule, timedir, kon, koff)